In [ ]:
!pip install transformers

# Предобученные под Python модели

В библиотеке Hugging Transformers есть несколько моделей, предобученных для кода на Python.
Все они работают в едином фреймворке этой библиотеки. Общий список моделей можно посмотреть по адресу: 
https://huggingface.co/models Я искал по ключевому слову CodeBERT

## CodeBERT от Microsoft

## Прогноз токенов по контексту (masked language modeling prediction)

Дается фрагмент кода, нужно предсказать маскированные токены.

In [17]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")
model.to(device)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [41]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

## Простой пример

Токеном <mask> обозначен неизвестный токен, который нужно предсказать.

In [42]:
CODE = """for i in my_range:\n print(<mask>)"""
print(CODE)

for i in my_range:
 print(<mask>)


Предсказываем токен с помощью fill_mask.

In [43]:
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)
outputs = fill_mask(CODE)

Выводим результаты с оценками вероятности.

In [44]:
for item in outputs:
    print(tokenizer.convert_ids_to_tokens(item['token']), item['score'])

i 0.9746569395065308
I 0.0033797631040215492
x 0.0016088365809991956
Ġi 0.0006835384410806
s 0.0006027265335433185


## Пример посложнее

Поскольку CodeBERT многоязычная языковая модель, интересно насколько хорошо она знает именно Python.

In [45]:
CODE = """from numpy.<mask> import svd"""
print(CODE)

from numpy.<mask> import svd


In [46]:
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)
outputs = fill_mask(CODE, top_k=10)

In [31]:
for item in outputs:
    print(tokenizer.convert_ids_to_tokens(item['token']), item['score'])

vd 0.3344379961490631
Ġlib 0.25087788701057434
Ġsparse 0.0348677858710289
stats 0.02802683413028717
norm 0.023705152794718742


На самом деле подразумевалось `from numpy.linalg import svd`.

## Доучивание модели

В принципе модель можно пробовать доучивать, т.к. обучать с нуля очень трудоемко. Можно
пробовать обучать как Roberta в библиотеке transformers, инициализируя готовой моделью. Нужно только задать дополнительный датасет с кодом только на Python см. https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb. Единственный момент: сама модель обучена как на парах код+документация к нему, так и на только коде. Тут возможно придется немного повозиться.

# Вычисление эмбеддингов

## Пример для простого кода

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

In [62]:
CODE = "from numpy.linalg import svd"
input_ids = torch.tensor(tokenizer.encode(CODE)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

In [63]:
last_hidden_states.shape

torch.Size([1, 12, 768])

In [64]:
last_hidden_states

tensor([[[-0.0650,  0.3750, -0.0022,  ..., -0.2516, -0.2930,  0.3319],
         [-0.4464,  0.1187,  0.1608,  ..., -0.8635, -0.9664,  0.2361],
         [-0.2363,  0.5910,  0.6816,  ..., -0.2713, -0.3131,  0.4560],
         ...,
         [-0.2245, -0.0632,  0.3228,  ..., -0.0709, -0.3570,  0.3159],
         [-0.1824,  0.1726,  0.4167,  ..., -0.7257, -0.2559,  0.1750],
         [-0.0641,  0.3753, -0.0021,  ..., -0.2514, -0.2935,  0.3315]]],
       grad_fn=<NativeLayerNormBackward>)

Аналогичный код для вычисления эмбеддингов в Tensorflow можно найти по ссылке https://github.com/huggingface/transformers/issues/1950

## Пример для кода посложнее

In [140]:
%%timeit
CODE = """yusuke_power = {"Yusuke Urameshi": "Spirit Gun"}
          hiei_power = {"Hiei": "Jagan Eye"}
          powers = dict()
          for dictionary in (yusuke_power, hiei_power):
             for key, value in dictionary.items():
                powers[key] = value
                powers = {key: value for d in (yusuke_power, hiei_power) for key, value in d.items()}
          powers = yusuke_power.copy()
          powers.update(hiei_power)
          powers = {**yusuke_power, **hiei_power}"""
input_ids = torch.tensor(tokenizer.encode(CODE)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]

67.2 ms ± 418 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
last_hidden_states.shape

torch.Size([1, 315, 768])

In [60]:
last_hidden_states

tensor([[[-0.2457,  0.0350, -0.1052,  ..., -0.1054, -0.5625,  0.4955],
         [-0.1148,  0.3186,  0.3916,  ..., -0.6488, -0.3554,  0.9227],
         [-0.1902,  0.4436,  0.3964,  ..., -1.1307, -0.3226,  1.0182],
         ...,
         [-0.2294,  0.1477,  0.6979,  ..., -1.2760, -0.4362,  0.6578],
         [-0.1939,  0.0437,  0.7932,  ..., -0.5501, -0.4389,  0.8314],
         [-0.2482,  0.0358, -0.1045,  ..., -0.1089, -0.5647,  0.4984]]],
       grad_fn=<NativeLayerNormBackward>)

## CodeBERT от Hugging Face

In [117]:
hugging_face_codeberts = ['huggingface/CodeBERTa-language-id', 'mrm8488/CodeBERTaPy', 'huggingface/CodeBERTa-small-v1']
hugging_face_codebert = 'huggingface/CodeBERTa-language-id'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained(hugging_face_codebert)
model = RobertaModel.from_pretrained(hugging_face_codebert)

Токеном <mask> обозначен неизвестный токен, который нужно предсказать.

In [131]:
CODE = """for i in my_range:\n print(<mask>)"""
print(CODE)

for i in my_range:
 print(<mask>)


Предсказываем токен с помощью fill_mask.

In [132]:
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)
outputs = fill_mask(CODE)

Выводим результаты с оценками вероятности.

In [133]:
for item in outputs:
    print(tokenizer.convert_ids_to_tokens(item['token']), item['score'])

( 0.13197968900203705
ĕ 0.08918817341327667
ity 0.055347152054309845
é 0.016465647146105766
cess 0.007545141503214836


## Пример посложнее

Интересно насколько хорошо модель знает именно Python.

In [134]:
CODE = """from numpy.<mask> import svd"""
print(CODE)

from numpy.<mask> import svd


In [135]:
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)
outputs = fill_mask(CODE, top_k=10)

In [136]:
for item in outputs:
    print(tokenizer.convert_ids_to_tokens(item['token']), item['score'])

( 0.03452527895569801
ĕ 0.011903248727321625
(" 0.01058276928961277
Ò 0.010515420697629452
ity 0.010480048134922981


На самом деле подразумевалось `from numpy.linalg import svd`.

Все 3 модели предсказывают заметно хуже, чем microsoft.

# Вычисление эмбеддингов

## Пример для простого кода

In [124]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained(hugging_face_codebert)
model = RobertaModel.from_pretrained(hugging_face_codebert)

In [125]:
CODE = "from numpy.linalg import svd"
input_ids = torch.tensor(tokenizer.encode(CODE)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

In [126]:
last_hidden_states.shape

torch.Size([1, 8, 768])

In [127]:
last_hidden_states

tensor([[[-0.5714,  0.0211,  0.1509,  ...,  0.4363,  0.1909, -0.7430],
         [-1.4592, -0.8622, -0.4383,  ...,  1.3121, -0.1740,  0.6295],
         [-0.7859, -0.1923, -0.2745,  ...,  1.5318,  0.6319, -1.6707],
         ...,
         [-0.0950, -1.2975,  0.4061,  ..., -0.2274, -0.7057,  0.7841],
         [ 0.3529, -0.1571, -0.3116,  ..., -0.2626,  0.4818, -0.8158],
         [-0.7207,  0.9594, -1.2018,  ...,  1.1717,  0.2522,  0.4499]]],
       grad_fn=<NativeLayerNormBackward>)

Аналогичный код для вычисления эмбеддингов в Tensorflow можно найти по ссылке https://github.com/huggingface/transformers/issues/1950

## Пример для кода посложнее

In [139]:
%%timeit
CODE = """yusuke_power = {"Yusuke Urameshi": "Spirit Gun"}
          hiei_power = {"Hiei": "Jagan Eye"}
          powers = dict()
          for dictionary in (yusuke_power, hiei_power):
             for key, value in dictionary.items():
                powers[key] = value
                powers = {key: value for d in (yusuke_power, hiei_power) for key, value in d.items()}
          powers = yusuke_power.copy()
          powers.update(hiei_power)
          powers = {**yusuke_power, **hiei_power}"""
input_ids = torch.tensor(tokenizer.encode(CODE)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]

67.9 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [129]:
last_hidden_states.shape

torch.Size([1, 193, 768])

In [130]:
last_hidden_states

tensor([[[ 1.1651,  1.0864,  0.2587,  ..., -1.0808, -0.0766, -0.4867],
         [ 1.3228, -0.8010,  0.0492,  ...,  0.2933,  2.0016,  0.8113],
         [ 0.6429, -0.0650,  0.8990,  ..., -0.1449,  1.5478, -0.1150],
         ...,
         [-0.2753,  0.6627,  1.2485,  ...,  1.1260,  0.8846,  1.1462],
         [ 0.6988,  0.1201,  0.2042,  ..., -1.0288, -1.6246,  0.4398],
         [ 0.4396,  1.4050,  0.2921,  ...,  1.7734, -1.3660, -0.4832]]],
       grad_fn=<NativeLayerNormBackward>)

По аналогичной схеме можно импортировать еще несколько fine-tuned языковых моделей, например, эту https://huggingface.co/mrm8488/codebert-base-finetuned-detect-insecure-code

## Выводы по найденным предобученным моделям

Время работы для вычисления эмбеддинга на одном сниппете кода составляет примерно 70мс, т.е. для 10000000 фрагментов нужно примерно 194 часа, чуть больше недели.